In [9]:
from mip import *

In [10]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 4, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

instanceBig1 = {
    "m" : 6,
    "n" : 9,
    "l" : [190, 185, 185, 190, 195, 185],
    "s" : [11, 11, 23, 16, 2, 1, 24, 14, 20],
    "distances" : [
        [0, 200, 119, 28, 180, 77, 145, 61, 123, 87],
        [200, 0, 81, 207, 38, 122, 55, 138, 76, 113],
        [119, 81, 0, 126, 69, 121, 26, 117, 91, 32],
        [28, 207, 126, 0, 187, 84, 152, 68, 130, 94],
        [170, 38, 79, 177, 0, 92, 58, 108, 46, 98],
        [77, 122, 121, 84, 102, 0, 100, 16, 46, 96],
        [145, 55, 26, 152, 58, 100, 0, 91, 70, 58],
        [61, 138, 113, 68, 118, 16, 91, 0, 62, 87],
        [123, 76, 91, 130, 56, 46, 70, 62, 0, 66],
        [87, 113, 32, 94, 94, 96, 58, 87, 66, 0]
    ]
}

In [11]:
instance = instanceBig1

In [12]:
n = instance["n"]
m = instance["m"]
s = instance["s"]
l = np.array(instance["l"])
distances = np.array(instance["distances"])
max_distance = np.max(distances, axis=1).sum()

# depot is n+1 (since 0-indexed it's n)

model = Model(solver_name=CBC)

# x[i,j,jk] = 1 if courier i at time k delivers package j
x = model.add_var_tensor((m, n+1, n+2), var_type=BINARY, name="x")

# y[i] is the totale distance travelled by courier i
y = model.add_var_tensor((m,), var_type=INTEGER, lb=0, ub=max_distance, name="y")

# t[i,j] is 1 if courier i delivers package j
t = model.add_var_tensor((m, n), var_type=BINARY, name="t")

# z[i, j1, j2] is 1 if courier i goes from package j1 to package j2
z = model.add_var_tensor((m, n+1, n+1), var_type=BINARY, name="z")

# each courier start from the depot
model += x[:, n, 0] == 1
# each courier end at the depot
model += x[:, n, n+1] == 1

# t[i,j] 
for i in range(m):
    for j in range(n):
        model += np.sum(x[i,j,:]) == t[i,j]

# every package is delivered
for j in range(n):
    model += np.sum(t[:,j]) == 1

# each courier can carry at most l[i] weight
for i in range(m):
    model += s @ t[i,:] <= l[i]

# z[i, j1, j2]
# z[i, j1, j2] true iff x[i, j1, k] and x[i, j2, k+1] are true for some k
for i in range(m):
    for j1 in range(n+1):
        for j2 in range(n+1):
            for k in range(n+1):
                model += z[i, j1, j2] <= x[i, j1, k]
                model += z[i, j1, j2] <= x[i, j2, k+1]
                model += z[i, j1, j2] >= x[i, j1, k] + x[i, j2, k+1] - 1

# copilot suggested this, maybe it's implicit in the other constraints
"""# each courier can only go from a package to another if he is delivering that package
for i in range(m):
    for j1 in range(n):
        for j2 in range(n):
            model += z[i, j1, j2] <= t[i, j1]
            model += z[i, j1, j2] <= t[i, j2]
            model += z[i, j1, j2] >= t[i, j1] + t[i, j2] - 1"""

# y[i]
for i in range(m):
    model += np.sum(distances * z[i,:,:]) == y[i]

# once a courier return to the depot, he can't go anywhere else
# x[i, n, k] = 1 => x[i, n, k+1] = 1
for i in range(m):
    for k in range(1,n+1):
        model += x[i, n, k] <= x[i, n, k+1]
    
v = model.add_var(name="v", var_type=INTEGER, lb=0, ub=max_distance)

for i in range(m):
    model += v >= y[i]

model.threads = -1

In [13]:
model.objective = minimize(v)

model.optimize()

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 60 small elements
Coin0506I Presolve 17403 (-744) rows, 1249 (-72) columns and 41796 (-1638) elements
Clp0030I 19 infeas 0.0022219071, obj 0.030750184 - mu 1.3709193e-06, its 105, 828 interior
Clp1000I sum of infeasibilities 0.000102838 - average 5.90922e-09, 426 fixed columns
Coin0506I Presolve 9063 (-8340) rows, 823 (-426) columns and 20450 (-21346) elements
Clp0006I 0  Obj 0.0389941 Primal inf 1.9414553e-05 (57) Dual inf 5.0028443e+13 (71)
Clp0029I End of values pass after 823 iterations
Clp0014I Perturbing problem by 0.001% of 1.2490919 - largest nonzero change 2.9877133e-05 ( 0.0014938567%) - largest zero change 2.9989116e-05
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - o

<OptimizationStatus.OPTIMAL: 0>

In [14]:
print([y[i].x for i in range(m)])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [15]:
print(model.objective_value)

0.0


In [16]:
for i in range(m):
    for j in range(n):
        if t[i,j].x == 1:
            print(f"Courier {i} delivers package {j}")

Courier 0 delivers package 0
Courier 0 delivers package 1
Courier 0 delivers package 2
Courier 0 delivers package 3
Courier 0 delivers package 4
Courier 0 delivers package 5
Courier 0 delivers package 6
Courier 0 delivers package 7
Courier 0 delivers package 8
